# Clean title column

### Title column is the purpose of the loan given by a user

In [2]:
import pandas as pd

accepted_df_v1 = pd.read_csv("/Users/abubakaral-faki/Documents/Data Project/MPV1/data/interim/v1-accepted.csv")

/var/folders/yj/v54tzbn14gb70ts6s0j5s3xm0000gn/T/ipykernel_1685/169166819.py:3: DtypeWarning: Columns (17,47,57,116,127,128,129,132,133,134,137,143,144,145) have mixed types. Specify dtype option on import or set low_memory=False.
  accepted_df_v1 = pd.read_csv("/Users/abubakaral-faki/Documents/Data Project/MPV1/data/interim/v1-accepted.csv")


In [4]:
print(accepted_df_v1['title'].info())
print(accepted_df_v1['title'].describe())

<class 'pandas.core.series.Series'>
RangeIndex: 2027964 entries, 0 to 2027963
Series name: title
Non-Null Count    Dtype 
--------------    ----- 
2004995 non-null  object
dtypes: object(1)
memory usage: 15.5+ MB
None
count                2004995
unique                 63154
top       Debt consolidation
freq                 1025175
Name: title, dtype: object


In [6]:
# get number of categories and Distribution of categores
accepted_df_v1['title'].value_counts()

title
Debt consolidation             1025175
Credit card refinancing         422086
Home improvement                120563
Other                           108712
Major purchase                   38895
                                ...   
Home buys                            1
Credit Card Family Recovery          1
help pay bill                        1
Credit Home Improvement              1
debt reduction/hone updates          1
Name: count, Length: 63154, dtype: int64

In [8]:
loan_purpose = accepted_df_v1['title'].value_counts().reset_index()

loan_purpose[:25]

,title,count
0,Debt consolidation,1025175
1,Credit card refinancing,422086
2,Home improvement,120563
3,Other,108712
4,Major purchase,38895
5,Medical expenses,21827
6,Business,18002
7,Car financing,17851
8,Debt Consolidation,15763
9,Vacation,12615


In [10]:
# Count Number of Missing Values
accepted_df_v1['title'].isna().sum()

22969

## Normalize titles

In [13]:
# Number of unique job titles before normalizing
print(len(accepted_df_v1['title'].unique()))

63155


In [15]:
# Strip Whitespace and normalize Job titles
accepted_df_v1['title'] = accepted_df_v1['title'].fillna("").apply(lambda x: str(x).lower().strip())

In [17]:
# Number of unique job titles after normalizing
print(len(accepted_df_v1['title'].unique()))

51792


## Save All Unique Loan titles for manual inspection

In [20]:
# Get uque Loan titles and their value counts
loan_titles = accepted_df_v1['title'].value_counts().reset_index()
loan_titles.columns = ['loan_purpose', 'count']

# Save unique loan titles to csv for manual inspection
loan_titles.to_csv('/Users/abubakaral-faki/Documents/Data Project/MPV1/temp_files/loan_titles_cleaned.csv', index = False)

# Redefining categories to reduce number of categories

In [23]:
# Change 'debt consolidation loan' category to 'debt consolidation'
accepted_df_v1.loc[accepted_df_v1['title'] == 'debt consolidation loan','title'] = 'debt consolidation'

In [25]:
# Change 'consolidation' & 'consolidate' category to 'consolidation loan'

consolidation_loans = ['consolidation', 'consolidate']
accepted_df_v1.loc[accepted_df_v1['title'].isin(consolidation_loans), 'title'] = 'consolidation loan'

In [27]:
# Combine 'credit card refinance' & 'credit card payoff' & 'credit card consolidation' into existing category 'credit card refinancing'

credit_card_loans = ['credit card refinance', 'credit card payoff', 'credit card consolidation', 'credit cards', 'credit consolidation',
                    'credit card refi', 'credit card debt', 'credit card pay off','cc consolidation', 'cc payoff', 
                     'credit card debt consolidation', 'cc loan', 'cc refinance', 'cc refi', 'credit payoff', 'cards', 'credit loan', 'credit card payment']
accepted_df_v1.loc[accepted_df_v1['title'].isin(credit_card_loans), 'title'] = 'credit card refinancing'

In [29]:
# Combine 'personal' category to existing category 'personal loan'
accepted_df_v1.loc[accepted_df_v1['title'].isin(['personal']), 'title'] = 'personal loan'

In [38]:
accepted_df_v1['title'].value_counts()[:20]

title
debt consolidation         1052511
credit card refinancing     438263
home improvement            123312
other                       108805
major purchase               39208
                             22969
medical expenses             21942
business                     18120
car financing                17878
consolidation loan           13023
vacation                     12824
moving and relocation        12024
home buying                  10762
personal loan                 5572
loan                          1636
payoff                        1463
debt                          1422
freedom                       1326
credit card                   1281
my loan                       1209
Name: count, dtype: int64

In [40]:
# Number of unique job titles after re-categorizing
print(len(accepted_df_v1['title'].unique()))

51770


There are appear to be numbers and dates which don't make sense. so we will be leaving them blank

## Remove pure numbers in loan title

In [44]:
accepted_df_v1 = accepted_df_v1[~accepted_df_v1['title'].str.match(r'\d+$', '', na = False)]

In [45]:
accepted_df_v1[~(accepted_df_v1['title'] == '')]['title'].sort_values(ascending = True)[:20]

1791260                                                "ccc"
1645481    "discover" ing i no longer want to "chase" aft...
1624103                     "for those that said i couldn't"
1830587                                           "freedom!"
1938497                               "funeral expenses loan
1624722                                    "get out of debt"
1789661                 "get out of debt" consolidation loan
1647738                      "getting ahead on my debt" loan
1811382             "getting over the hump" debt consolidatn
1651661                                          "good loan"
1840308                          "good to great" investments
1906389                                   "goodbye apr high"
1784591                                "happy if successful"
1629555                                    "happy to return"
1830685                                          "moving on"
1760463                                "my home improvement"
1799943                 

## Identify debt consolidation loans

In [47]:
# Remove all spaces from title columns
accepted_df_v1['c_title'] = accepted_df_v1['title'].str.strip().str.replace("_", "", regex=False)

In [48]:
accepted_df_v1['c_title'].value_counts()[:20]

c_title
debt consolidation         1052511
credit card refinancing     438263
home improvement            123312
other                       108805
major purchase               39208
                             22969
medical expenses             21942
business                     18120
car financing                17878
consolidation loan           13023
vacation                     12824
moving and relocation        12024
home buying                  10762
personal loan                 5572
loan                          1636
payoff                        1464
debt                          1422
freedom                       1326
credit card                   1281
my loan                       1209
Name: count, dtype: int64

In [89]:
from thefuzz import process
import time

# Remove titles tha contain the word card so we don't match for Credit card Consolidate
accepted_df_debt = accepted_df_v1[~accepted_df_v1['c_title'].str.contains(r"card|cc|credit|c/c|c.c.|c.c", case=False, na=False)]
accepted_df_debt = accepted_df_v1[accepted_df_v1['c_title'].str.contains("debt", case=False, na=False)]

# Start Timer
start_timer = time.time()

# Get unique titles containing debt
unique_loans = accepted_df_debt['c_title'].dropna().unique()
print('ul: ', unique_loans[:5])


# Define title to match
match_title = 'debt consolidation'
print(f"Finding loan titles close to {match_title}")

# Find matches close to match_title
matches = process.extract(match_title, unique_loans, limit = len(unique_loans))
print('matches: ', matches[:5])


# Export matches to csv for manual inspection
file_path = f'/Users/abubakaral-faki/Documents/Data Project/MPV1/temp_files/{match_title}.csv'
matches_df = pd.DataFrame(matches, columns = ['loan_title', 'similarity_score'])
matches_df.to_csv(file_path, index = False)


print("")
print(f'{match_title}.csv file saved for manual inspection')

print(matches_df.head())

# === 🔹 Optimized Dictionary Mapping ===
# Create a mapping dictionary to replace incorrect titles with 'debtconsolidation'
correction_map = {title: match_title for title in matches_df['loan_title']}

# print('correction Map:', correction_map)

# Efficiently update values using `.map()`
accepted_df_v1.loc[accepted_df_v1['c_title'].isin(correction_map.keys()), 'c_title'] = accepted_df_v1['c_title'].map(correction_map)

# End Timer
end_timer = time.time()

print(f"\n✅ Mapping Applied in {round(end_timer - start_timer, 2)} seconds")

ul:  ['debt consolidation' 'debtc' 'debt payoff' 'debt pay off' 'debt clean up']
Finding loan titles close to debt consolidation
matches:  [('debt consolidation', 100), ('debt-consolidation', 100), ('debt consolidation..', 100), ('debt consolidation.', 100), ('debt consolidation-', 100)]

debt consolidation.csv file saved for manual inspection
             loan_title  similarity_score
0    debt consolidation               100
1    debt-consolidation               100
2  debt consolidation..               100
3   debt consolidation.               100
4   debt consolidation-               100

✅ Mapping Applied in 0.24 seconds


Some title like 'consolidation' will be considered as 'debt consolidation'

Some titles like 'debt' will be considered 'debt consolidation'

Some titles like 'debtsucks','loan4debt', 'debtclearence', 'paybakdebt', 'lookingtogetdebtfree' will be considered 'debt consolidation', 

Dual purpose loans like 'medicalandothermiscdebt' and 'homerepairanddebtpayoff' will be considered debt consolidation

There will be erros while cleaning because Fuzzyy matching is not context and menaing aware

In [92]:
# Check Value counts
accepted_df_v1['c_title'].value_counts()[:20]

c_title
debt consolidation         1076622
credit card refinancing     458335
home improvement            123312
other                       108805
major purchase               39208
medical expenses             23652
                             22969
business                     18182
car financing                17878
consolidation loan           13023
vacation                     12824
moving and relocation        12024
home buying                  10762
personal loan                 5572
loan                          1636
payoff                        1464
freedom                       1326
my loan                       1209
green loan                    1067
home improvement loan          777
Name: count, dtype: int64

In [94]:
# Number of unique value counts
print(len(accepted_df_v1['c_title'].unique()))

35648


# Identify all loan titles with cc and change them to 'credit card refinancing' category

Why?
 - I would like to reduce the number of categories in the loan_title column for more a more accurate risk model

In [97]:
from thefuzz import process

unique_loan_titles = accepted_df_v1['c_title'].dropna().unique()

find_keyword = 'cc'

matches = process.extract(find_keyword, unique_loan_titles, limit = len(unique_loan_titles))

# Convert to DataFrame
matches_df = pd.DataFrame(matches, columns = ['loan_title', 'similarity_score'])

#Export matches_df to .csv file
file_path = '/Users/abubakaral-faki/Documents/Data Project/MPV1/temp_files/cc_loans.csv'
matches_df.to_csv(file_path, index = False) # Score of 86 seems to be the best threshold


In [99]:
# Dictionary Map
matches_df_86 = matches_df[matches_df['similarity_score'] >= 86]

# remove 'transacct' from list
matches_df_86 = matches_df_86[matches_df_86['loan_title'] != 'transacct']

#correction_map to chage titles with 'c
credit_correction_map = {mismatched_title: 'credit card refinancing' for mismatched_title in matches_df_86['loan_title']}


# Efficiently update values using `.map()`
accepted_df_v1.loc[accepted_df_v1['c_title'].isin(credit_correction_map.keys()), 'c_title'] = accepted_df_v1['c_title'].map(credit_correction_map)

print(accepted_df_v1['c_title'].value_counts())

c_title
debt consolidation                         1076622
credit card refinancing                     459000
home improvement                            123312
other                                       108805
major purchase                               39208
                                            ...   
getting away from washington mutual              1
ebay business expansion                          1
reducing interest pmnts on high apr ccs          1
350 rich. terrace                                1
takeitaway                                       1
Name: count, Length: 35383, dtype: int64


## Identify all loan titles with the word credit and change them to 'credit card refinancing' category

In [102]:
keyword = 'credit'

unique_loan_titles = accepted_df_v1['c_title'].dropna().unique()

matches = process.extract(keyword, unique_loan_titles, limit = len(unique_loan_titles))

matches_df = pd.DataFrame(matches, columns = ['loan_title', 'similarity_score'])

#Export to csv
file_path = '/Users/abubakaral-faki/Documents/Data Project/MPV1/temp_files/credit_card_loan.csv'
matches_df.to_csv(file_path, index = False)

#Filer for titled with a score equal to or greater than 71
matched_df_filtered = matches_df[matches_df['similarity_score'] >= 71]

# Remove the false posittives
matched_df_filtered = matched_df_filtered[~matched_df_filtered['loan_title'].isin(['incredible','redirect of funds', 'pay dedit', 'car'])]

# Correction Map
credit_correction_map = {mismatched_title: 'credit card refinancing' for mismatched_title in matched_df_filtered['loan_title']}

# Apply Correction map 
accepted_df_v1.loc[accepted_df_v1['c_title'].isin(credit_correction_map.keys()), 'c_title'] = accepted_df_v1['c_title'].map(credit_correction_map)

print(accepted_df_v1['c_title'].value_counts())

c_title
debt consolidation                          1076622
credit card refinancing                      460062
home improvement                             123312
other                                        108805
major purchase                                39208
                                             ...   
2007tax                                           1
help pay the bills                                1
to finance tax payments                           1
payng off higher rate at fixed rate/term          1
takeitaway                                        1
Name: count, Length: 34909, dtype: int64


In [104]:
# Check number of unique categories
print('Number of Unique categries: ', len(accepted_df_v1['c_title'].unique()))

Number of Unique categries:  34909


In [106]:
keyword = 'card'

unique_loan_titles = accepted_df_v1['c_title'].dropna().unique()

matches = process.extract(keyword, unique_loan_titles, limit = len(unique_loan_titles))

matches_df = pd.DataFrame(matches, columns = ['loan_title', 'similarity_score'])

#Export to csv
file_path = '/Users/abubakaral-faki/Documents/Data Project/MPV1/temp_files/card_loan.csv'
matches_df.to_csv(file_path, index = False)

#Filer for titles with a score equal to or greater than 89
matched_df_filtered = matches_df[matches_df['similarity_score'] >= 89]

# Remove the false posittive
matched_df_filtered = matched_df_filtered[~matched_df_filtered['loan_title'].isin(['ar'])]

# Correction Map
credit_correction_map = {mismatched_title: 'credit card refinancing' for mismatched_title in matched_df_filtered['loan_title']}

# Apply Correction map 
accepted_df_v1.loc[accepted_df_v1['c_title'].isin(credit_correction_map.keys()), 'c_title'] = accepted_df_v1['c_title'].map(credit_correction_map)

print(accepted_df_v1['c_title'].value_counts()[:15])

c_title
debt consolidation         1076622
credit card refinancing     461012
home improvement            123312
other                       108805
major purchase               39208
medical expenses             23652
                             22969
business                     18182
car financing                17878
consolidation loan           13023
vacation                     12824
moving and relocation        12024
home buying                  10762
personal loan                 5572
loan                          1636
Name: count, dtype: int64


In [108]:
print('Number of Unique categories: ', len(accepted_df_v1['c_title'].unique()))

Number of Unique categories:  34373


## Identify home Improvent loans

In [111]:
from thefuzz import process
import time

#Make sure the word hom
home_improvement = accepted_df_v1[accepted_df_v1['c_title'].str.contains(r'improvement|home', case = False, na = False)]
home_improvement = accepted_df_v1[~accepted_df_v1['c_title'].str.contains(r'loan|refinance|repair|buy|building|bills|new', case = False, na = False)]

# Start Timer
start_timer = time.time()

# Get unique titles
unique_loans = home_improvement['c_title'].dropna().unique()

# Define title to match
match_title = 'home improvement'
print(f"Finding loan titles close to {match_title}")

# Find matches close to match_title
matches = process.extract(match_title, unique_loans, limit = len(unique_loans))

# Export matches to csv for manual inspection
file_path = f'/Users/abubakaral-faki/Documents/Data Project/MPV1/temp_files/{match_title}1111.csv'
matches_df = pd.DataFrame(matches, columns = ['loan_title', 'similarity_score'])
matches_df.to_csv(file_path, index = False)

print(matches_df.head())

# Remove False Positives
false_positives = ['primaryhomeinvestment']
matches_df = matches_df[~matches_df['loan_title'].isin([false_positives])]

# Filter for matching with a score of 60 and above and that have the word debt in them
filtered_matches_df = matches_df[matches_df['similarity_score'] >= 87]

# === 🔹 Optimized Dictionary Mapping ===
# Create a mapping dictionary to replace incorrect titles with 'debtconsolidation'
correction_map = {title: 'home improvement' for title in filtered_matches_df['loan_title']}

# Efficiently update values using `.map()`
accepted_df_v1.loc[accepted_df_v1['c_title'].isin(correction_map.keys()), 'c_title'] = accepted_df_v1['c_title'].map(correction_map)

# End Timer
end_timer = time.time()

# End Timer
end_timer = time.time()

print(f"\n✅ Mapping Applied in {round(end_timer - start_timer, 2)} seconds")

Finding loan titles close to home improvement
            loan_title  similarity_score
0     home improvement               100
1  home improvement...               100
2    home improvement.               100
3    home improvements                97
4    home  improvement                97

✅ Mapping Applied in 0.45 seconds


In [112]:
print(accepted_df_v1['c_title'].value_counts()[:70])

c_title
debt consolidation         1076622
credit card refinancing     461012
home improvement            125167
other                       108805
major purchase               39208
                            ...   
lc loan                        117
taxes                          117
bill payoff                    115
money                          114
motorcycle loan                111
Name: count, Length: 70, dtype: int64


In [113]:
print('Number of Unique categories: ', len(accepted_df_v1['c_title'].unique()))

Number of Unique categories:  33889


## Identify 'medical' titles

In [118]:
## Identify medical

from thefuzz import process
import time
#Filter accepted_df_v1 for medical bills
medical_loans = accepted_df_v1[accepted_df_v1['c_title'].str.contains(r'medic|medical|medic|med|medx', case = False, na = False)]

# Start Timer
start_timer = time.time()

# Get unique titles
unique_loans = medical_loans['c_title'].dropna().unique()

# Define title to match
match_title = 'medical expense'
print(f"Finding loan titles close to {match_title}")

# Find matches close to match_title
matches = process.extract(match_title, unique_loans, limit = len(unique_loans))
print(matches[:5])

# Export matches to csv for manual inspection
file_path = f'/Users/abubakaral-faki/Documents/Data Project/MPV1/temp_files/{match_title}.csv'
matches_df = pd.DataFrame(matches, columns = ['loan_title', 'similarity_score'])
matches_df.to_csv(file_path, index = False)

#Remove False Positives
false_positives = ['goodbye homedepot', 'goodbye homedepot', 'reformed unhappy shopper', 'reformed','xelamed','odymeds', 'homedeposit', 'remedy','confirmed', 'homedepot'
                   'discover/homedepot consolidation', 'ahmed', 'overwhelmed by bills', 'stanford medstudent seekinginvestment$$', '20th anniversary mediterranean cruise',
                   'i was scammed and now recovering','nchomedream', 'someday i will pay it forward - thanks', 'i dreamed a dream','king media', 'home "remediation',
                   'rapidly growing internet media business', 'high interest remediation', 'dept remediation', 'need car for transportation immediately',
                   'immediate repayment for a personal loan', 'new way yoga and meditation', 'moving loan for immediate moving costs', 'paramedic school loan (ncti)',
                   'business loan for media design startup','remote site paramedicine contractor', 'immediate loan relief','paramedic school', 'multimedia  expansion',
                   'bofa,homedept,tax', 'paysomedepts', 'stress remedy & dream propellant', 'from a small to medium size business', 'remote site paramedicine contractor',
                   'paysomedepts'
                  ]
filtered_matches_df = matches_df[~matches_df['loan_title'].isin(false_positives)]

# === 🔹 Optimized Dictionary Mapping ===
# Create a mapping dictionary to replace incorrect titles with 'debtconsolidation'
correction_map = {title: 'medical expenses' for title in filtered_matches_df['loan_title']}

# Efficiently update values using `.map()`
accepted_df_v1.loc[accepted_df_v1['c_title'].isin(correction_map.keys()), 'c_title'] = accepted_df_v1['c_title'].map(correction_map)

# End Timer
end_timer = time.time()

# End Timer
end_timer = time.time()

print(f"\n✅ Mapping Applied in {round(end_timer - start_timer, 2)} seconds")

Finding loan titles close to medical expense
[('medical expenses', 97), ('ahmed', 68), ('multimedia  expansion', 61), ('remedy', 60), ('king media', 60)]

✅ Mapping Applied in 0.09 seconds


In [119]:
print('Number of Unique categories: ', len(accepted_df_v1['c_title'].unique()))

Number of Unique categories:  33889


In [120]:
print(accepted_df_v1['c_title'].value_counts()[:30])

c_title
debt consolidation         1076622
credit card refinancing     461012
home improvement            125167
other                       108805
major purchase               39208
medical expenses             23652
                             22969
business                     18182
car financing                17878
consolidation loan           13023
vacation                     12824
moving and relocation        12024
home buying                  10762
personal loan                 5572
loan                          1636
payoff                        1464
freedom                       1326
my loan                       1209
green loan                    1067
home improvement loan          777
refinance                      691
pay off                        681
wedding                        668
bill consolidation             585
wedding loan                   577
small business loan            524
car loan                       505
bills                          487
loan consoli

In [122]:
loan_purpose = accepted_df_v1['title'].value_counts().reset_index()
loan_purpose.to_csv('/Users/abubakaral-faki/Documents/Data Project/MPV1/temp_files/loan_titles_extraclean.csv', index = False)

# Clean titles Fuzzy Matching didn't Identify

In [127]:
# Get all unique titles after cleaning
semi_cleaned_titles = accepted_df_v1['c_title'].value_counts().reset_index()
semi_cleaned_titles.columns = ['loan_purpose', 'count']

# Save unique_titles_cleaned
file_path = '/Users/abubakaral-faki/Documents/Data Project/MPV1/temp_files/semi_cleaned_titles.csv'
semi_cleaned_titles.to_csv(file_path, index = False)

In [129]:
# Identify Loans used to to help with credit card debt
credit_card_loans = semi_cleaned_titles[semi_cleaned_titles['loan_purpose'].str.contains(r'\bcc\b|\bcard\b|\bcredit\b|\bccr\b', case = False, na = False, regex = True)]

file_path = '/Users/abubakaral-faki/Documents/Data Project/MPV1/temp_files/credit_cards_left.csv'
credit_card_loans.to_csv(file_path, index = False)

In [131]:
# Dictionary map to change titles of credit card related loans to 'credit card refinancing'
credit_map = {credit_title: 'credit card refinancing' for credit_title in credit_card_loans['loan_purpose']}

#Effiency replace titles of credit card related loans to 'credit card refinancing' using vectrorized operations
accepted_df_v1.loc[accepted_df_v1['c_title'].isin(credit_map.keys()), 'c_title'] = accepted_df_v1['c_title'].map(credit_map)

In [133]:
print('Number of Unique categories: ', len(accepted_df_v1['c_title'].unique()))

Number of Unique categories:  33889


# Identify Business Loans

In [136]:
# Get all unique titles after cleaning
semi_cleaned_titles = accepted_df_v1['c_title'].value_counts().reset_index()
semi_cleaned_titles.columns = ['loan_purpose', 'count']

# Save unique_titles_cleaned
#file_path = '/Users/abubakaral-faki/Documents/Data Project/MPV1/temp_files/semi_cleaned_titles.csv'
#semi_cleaned_titles.to_csv(file_path, index = False)

In [138]:
# Identify Loans used to to help with credit card debt
business_loans = semi_cleaned_titles[semi_cleaned_titles['loan_purpose'].str.contains(r'bussiness|biz|\busines', case = False, na = False, regex = True)]

file_path = '/Users/abubakaral-faki/Documents/Data Project/MPV1/temp_files/business_loans_left.csv'
business_loans.to_csv(file_path, index = False)

In [140]:
# Dictionary map to change titles of credit card related loans to 'credit card refinancing'
business_map = {business_title: 'business' for business_title in business_loans['loan_purpose']}

#Effiency replace titles of credit card related loans to 'credit card refinancing' using vectrorized operations
accepted_df_v1.loc[accepted_df_v1['c_title'].isin(business_map.keys()), 'c_title'] = accepted_df_v1['c_title'].map(business_map)

In [145]:
print('Number of Unique categories: ', len(accepted_df_v1['c_title'].unique()))

Number of Unique categories:  33889


In [155]:
accepted_df_v1['c_title'].info()

<class 'pandas.core.series.Series'>
Index: 2027841 entries, 0 to 2027963
Series name: c_title
Non-Null Count    Dtype 
--------------    ----- 
2027841 non-null  object
dtypes: object(1)
memory usage: 95.4+ MB


# Export cleaned Dataframe

In [161]:
file_path = '/Users/abubakaral-faki/Documents/Data Project/MPV1/data/processed/accepted_df_v2.csv'
accepted_df_v1.to_csv(file_path, index = False)

print('Dataframe successfully saved as accepted_df_v2.csv')

Dataframe successfully saved as accepted_df_v2.csv
